In [11]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

%matplotlib inline

In [12]:
df = pd.read_csv('/Users/menusha/Desktop/sentiment_data.csv')
df = df[pd.notnull(df['tags'])]
df

,post,tags
0,Visual Studio has a lot of features. The firs...,VisualStudio
1,Users have to spend high cost for Enterprise L...,VisualStudio
2,I've used various flavors of Eclipse and Intel...,VisualStudio
3,There's really not much to dislike about Visua...,VisualStudio
4,Microsoft visual studio is developing software...,VisualStudio
5,The code compilation error messages are someti...,VisualStudio
6,IntelliJ-IDEA It is a Java integrated software...,IntelliJIdea
7,Intellij is one of the Best IDE available for ...,IntelliJIdea
8,In my experience this tool is a great and must...,IntelliJIdea
9,It intensifies the work load upon the CPU esp...,IntelliJIdea


In [13]:
df['post'].apply(lambda x: len(x.split(' '))).sum()


1984

In [15]:
def print_plot(index):
    example = df[df.index == index][['post', 'tags']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])

In [16]:
print_plot(10)


What I like most are the shortcuts. Every other IDE also has shortcuts but IntelliJ (and Jetbrains generally) has done a great job with theirs. At least for men, they seem intuitive. - Offers many, many features but still manages to have a great performance. - Sounds trivial, but the settings are well structured and searchable! Almost everything is customizable and if not there is probably a plugin for want you want it to do. - I do a lot of web-programming so I appreciate small little features like native less, sass support, live reloading or even showing the color next to a color code in CSS stylesheets. Even though Intellij should be Java focused, this also applies for Javascript and its complex ecosystem (I used React, JSX, Angular, Elm). - Solid Versioning Support with git as well as svn, both within the console and the UI. - Great Maven support. Auto-save, great search function. Overall I could list all the features I like, many you can find in other IDEs as well. But when using 

In [17]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [18]:
df['post'] = df['post'].apply(clean_text)

In [19]:
print_plot(10)

like shortcuts every ide also shortcuts intellij jetbrains generally done great job least men seem intuitive offers many many features still manages great performance sounds trivial settings well structured searchable almost everything customizable probably plugin want want lot webprogramming appreciate small little features like native less sass support live reloading even showing color next color code css stylesheets even though intellij java focused also applies javascript complex ecosystem used react jsx angular elm solid versioning support git well svn within console ui great maven support autosave great search function overall could list features like many find ides well using intellij feel gets way lets work dont know something usually nice intelligent trick hard find online student academic license really appreciated
Tag: IntelliJIdea


In [20]:
df['post'].apply(lambda x: len(x.split(' '))).sum()

1084

In [21]:
X = df.post
y = df.tags
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [24]:
my_tags = ['VisualStudio','IntelliJIdea','netbeans']

In [25]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.3333333333333333
              precision    recall  f1-score   support

VisualStudio       0.25      1.00      0.40         1
IntelliJIdea       0.00      0.00      0.00         4
    netbeans       0.50      1.00      0.67         1

   micro avg       0.33      0.33      0.33         6
   macro avg       0.25      0.67      0.36         6
weighted avg       0.12      0.33      0.18         6

CPU times: user 14 ms, sys: 3.79 ms, total: 17.8 ms
Wall time: 15.7 ms


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
